In [1]:
import networkx as nx
import tarfile
import os

# --- CONFIGURAZIONE ---
# Percorso del file .tgz dentro la tua cartella 'got'
archive_path = os.path.join("got", "GoT_dyn_ts40.tgz") 
output_filename = "GoT_S1_S2.graphml"

print(f"--- INIZIO ESTRAZIONE DA: {archive_path} ---")

# Verifichiamo che il file esista prima di partire
if not os.path.exists(archive_path):
    print(f"❌ Errore: Non trovo il file in {archive_path}")
    print("Assicurati che la cartella si chiami 'got' (minuscolo) e il file sia dentro.")
else:
    # Creiamo un grafo vuoto per accumulare i dati
    G_final = nx.Graph()
    
    # Apriamo l'archivio .tgz in modalità lettura
    with tarfile.open(archive_path, "r:gz") as tar:
        # Otteniamo la lista di tutti i file contenuti
        files = tar.getnames()
        
        # FILTRO: Cerchiamo solo i file che hanno "S01" o "S02" nel nome
        # I file di solito si chiamano tipo 'GoT_S01_EP01_...'
        target_files = [f for f in files if "S01" in f or "S02" in f]
        target_files.sort() # Ordiniamo cronologicamente
        
        print(f"Trovati {len(target_files)} segmenti temporali per le Stagioni 1 e 2.")
        
        count = 0
        for member_name in target_files:
            # Estraiamo il singolo file in memoria (senza salvarlo su disco)
            f = tar.extractfile(member_name)
            if f is not None:
                # Leggiamo il grafo parziale di quel momento
                G_temp = nx.read_graphml(f)
                
                # --- AGGREGAZIONE (SOMMA DEI PESI) ---
                for u, v, data in G_temp.edges(data=True):
                    # Se non c'è un peso esplicito, contiamo 1 interazione
                    weight = data.get('weight', 1.0)
                    
                    # Se l'arco esiste già nel grafo finale, sommiamo il peso
                    if G_final.has_edge(u, v):
                        G_final[u][v]['weight'] += weight
                    else:
                        # Se è la prima volta che si parlano, creiamo l'arco
                        G_final.add_edge(u, v, weight=weight)
                
                # Aggiungiamo i nodi (per mantenere attributi se presenti)
                for node, attr in G_temp.nodes(data=True):
                    if node not in G_final:
                        G_final.add_node(node, **attr)
                
                count += 1
                # Stampiamo un progresso ogni 50 file per non intasare lo schermo
                if count % 50 == 0:
                    print(f"Processati {count}/{len(target_files)}...")

    # --- SALVATAGGIO ---
    # Salviamo il risultato finale nella cartella principale del progetto
    nx.write_graphml(G_final, output_filename)
    
    print(f"\n✅ SUCCESSO! File creato: {output_filename}")
    print(f"📊 Statistiche S1+S2:")
    print(f"   - Nodi (Personaggi): {G_final.number_of_nodes()}")
    print(f"   - Archi (Relazioni): {G_final.number_of_edges()}")
    print("Ora puoi aprire questo file con Gephi o caricarlo con 'nx.read_graphml'!")

--- INIZIO ESTRAZIONE DA: got\GoT_dyn_ts40.tgz ---
Trovati 446 segmenti temporali per le Stagioni 1 e 2.
Processati 50/446...
Processati 100/446...
Processati 150/446...
Processati 200/446...
Processati 250/446...
Processati 300/446...
Processati 350/446...
Processati 400/446...

✅ SUCCESSO! File creato: GoT_S1_S2.graphml
📊 Statistiche S1+S2:
   - Nodi (Personaggi): 338
   - Archi (Relazioni): 494
Ora puoi aprire questo file con Gephi o caricarlo con 'nx.read_graphml'!


### CORREZIONE NOMI

In [3]:
import networkx as nx

# 1. Carichiamo il grafo "Master" (quello che ha i nomi giusti)
G_master = nx.read_graphml("got/GoT_cum.graphml")
# Creiamo un dizionario {id: "Nome Vero"} (es. {'n0': 'Jon Snow'})
id_to_name = nx.get_node_attributes(G_master, 'label')

# 2. Carichiamo il tuo grafo S1-S2 (quello con i nomi rotti)
G_s1_s2 = nx.read_graphml("GoT_S1_S2.graphml")

print("--- RIPARAZIONE NOMI ---")
print(f"Prima: Nodo di esempio -> {list(G_s1_s2.nodes())[0]}")

# 3. Applichiamo i nomi corretti
# NetworkX ha una funzione apposta per rinominare i nodi usando un dizionario
# Copiamo solo i nodi che esistono in S1-S2
mapping = {k: v for k, v in id_to_name.items() if k in G_s1_s2.nodes()}
G_fixed = nx.relabel_nodes(G_s1_s2, mapping)

# 4. Salviamo il file corretto sovrascrivendo il vecchio
nx.write_graphml(G_fixed, "GoT_S1_S2.graphml")

print(f"Dopo: Nodo di esempio -> {list(G_fixed.nodes())[0]}")
print("✅ File riparato! Ora puoi riaprirlo su Gephi.")

--- RIPARAZIONE NOMI ---
Prima: Nodo di esempio -> n4
Dopo: Nodo di esempio -> Theon Greyjoy
✅ File riparato! Ora puoi riaprirlo su Gephi.
